In [155]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [156]:
train = pd.read_csv("train.csv")
leaderboard = pd.read_csv("leaderboard_dataset.csv")
test = pd.read_csv("test.csv")

In [157]:
train = train.fillna(0)
leaderboard = leaderboard.fillna(0)
test = test.fillna(0)

In [158]:
y = train["VAR21"]
X = train.drop(["VAR21"], axis=1)

In [159]:
from sklearn.preprocessing import LabelEncoder

for c in X.columns:
    if X[c].dtype == 'object':
        lbl = LabelEncoder() 
        lbl.fit(list(X[c].values) + list(test[c].values) + list(leaderboard[c].values)) 
        X[c] = lbl.transform(list(X[c].values))
        test[c] = lbl.transform(list(test[c].values))
        leaderboard[c] = lbl.transform(list(leaderboard[c].values))

In [160]:
from copy import deepcopy
lbl = LabelEncoder() 
lbl.fit(list(y.values)) 
y = lbl.transform(list(y))

In [161]:
y

array([1, 0, 2, ..., 1, 1, 0])

In [162]:
from sklearn.model_selection import train_test_split
from sklearn.metrics.scorer import make_scorer


def my_custom_accuracy(y_true, y_pred):
    total=0
    for t,p in zip(y_true, y_pred):
        if(t==p):
            total+=1000
        if(t==0 and p==1):
            total-=100
        if(t==0 and p==2):
            total-=50
        if(t==2 and p==1):
            total-=50
        if(t==2 and p==0):
            total-=100
        if(t==1 and p==2):
            total-=100
        if(t==1 and p==0):
            total-=200
    return total

# Make a custom a scorer from the custom metric function
# Note: greater_is_better=False in make_scorer below would mean that the scoring function should be minimized.
my_custom_scorer = make_scorer(my_custom_accuracy, greater_is_better=True)
X, X_test, y, y_test = train_test_split(X, y, train_size=30000, test_size=4000)

#tpot = TPOTClassifier(generations=10, population_size=20, verbosity=3, scoring=my_custom_scorer, cv=3)

In [163]:
from sklearn.ensemble import GradientBoostingClassifier

In [164]:
from sklearn.model_selection import KFold

In [165]:
kf = KFold(n_splits=3)

In [171]:
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.values[train_index], X.values[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    y1_train = deepcopy(y_train)
    y2_train = deepcopy(y_train)
    y3_train = deepcopy(y_train)
    
    y1_test = deepcopy(y_test)
    y2_test = deepcopy(y_test)
    y3_test = deepcopy(y_test)
    
    np.place(y1_test, y1_test == 0, 1)
    np.place(y2_test, y2_test == 1, 2)
    np.place(y3_test, y3_test == 2, 0)
    
    np.place(y1_train, y1_train == 0, 1)
    np.place(y2_train, y2_train == 1, 2)
    np.place(y3_train, y3_train == 2, 0)
    
    model1 = GradientBoostingClassifier()
    model2 = GradientBoostingClassifier()
    model3 = GradientBoostingClassifier()
    
    model1.fit(X_train, y1_train)
    model2.fit(X_train, y2_train)
    model3.fit(X_train, y3_train)
    
    print(model1.predict_proba(X_test)[:,1])
    print(model1.predict(X_test))
    print(model2.predict_proba(X_test)[:,1])
    print(model2.predict(X_test))
    print(model3.predict_proba(X_test)[:,1])
    print(model3.predict(X_test))
    

TRAIN: [10000 10001 10002 ... 29997 29998 29999] TEST: [   0    1    2 ... 9997 9998 9999]
[0.74087114 0.6149512  0.55529872 ... 0.64084579 0.47879445 0.46202789]
[2 2 2 ... 2 1 1]
[0.88668705 0.8751275  0.89638353 ... 0.89984859 0.9379581  0.73086167]
[2 2 2 ... 2 2 2]
[0.15728916 0.21780285 0.33911813 ... 0.21590854 0.38018371 0.18535847]
[0 0 0 ... 0 0 0]
TRAIN: [    0     1     2 ... 29997 29998 29999] TEST: [10000 10001 10002 ... 19997 19998 19999]
[0.61508622 0.53905222 0.66735603 ... 0.65853502 0.69467707 0.54779539]
[2 2 2 ... 2 2 2]
[0.87986991 0.89874816 0.9089907  ... 0.91957538 0.91146743 0.88936958]
[2 2 2 ... 2 2 2]
[0.20986506 0.33621489 0.19753577 ... 0.28041084 0.21827254 0.30857652]
[0 0 0 ... 0 0 0]
TRAIN: [    0     1     2 ... 19997 19998 19999] TEST: [20000 20001 20002 ... 29997 29998 29999]
[0.54999148 0.60799109 0.48300659 ... 0.58713728 0.59428211 0.703447  ]
[2 2 1 ... 2 2 2]
[0.89789153 0.92571007 0.93101068 ... 0.90234841 0.91659666 0.9109534 ]
[2 2 2 ... 2 

In [149]:
y[10000:29999]

array([2, 0, 1, ..., 2, 0, 2])

In [99]:
y_train_split.shape

(30000,)